In [1]:
import cloudknot as ck

In [2]:
def dki_dti_cod(params):
    subject, hcp_ak, hcp_sk = params
    import numpy as np
    import pandas as pd
    import s3fs
    import json
    import logging
    import os.path as op
    import nibabel as nib
    import dipy.reconst.dti as dti
    import dipy.reconst.dki as dki
    import dipy.core.gradients as dpg
    import dipy.reconst.cross_validation as xval
    
    import AFQ.data as afd 
    import AFQ.registration as reg
    logging.basicConfig(level=logging.INFO)
    log = logging.getLogger(__name__)    
    
    log.info(f"Fetching HCP subject {subject}")
    afd.fetch_hcp([subject], 
                  profile_name=False,
                  aws_access_key_id=hcp_ak,
                  aws_secret_access_key=hcp_sk)    
        
    dwi_dir = op.join(afd.afq_home, 'HCP', 'derivatives',
                      'dmriprep', f'sub-{subject}', 'sess-01/dwi')

    anat_dir = op.join(afd.afq_home, 'HCP', 'derivatives',
                      'dmriprep', f'sub-{subject}', 'sess-01/anat')

    hardi_fdata = op.join(dwi_dir, f"sub-{subject}_dwi.nii.gz")
    hardi_fbval = op.join(dwi_dir, f"sub-{subject}_dwi.bval")
    hardi_fbvec = op.join(dwi_dir, f"sub-{subject}_dwi.bvec")
    
    log.info(f"Reading data from file {hardi_fdata}")
    dwi_img = nib.load(hardi_fdata)
    dwi_data = dwi_img.get_fdata()

    log.info(f"Creating gradient table from {hardi_fbval} and {hardi_fbvec}")
    gtab = dpg.gradient_table(hardi_fbval, hardi_fbvec)
    
    bucket_name = 'dipy.dki.hcp'
    fs = s3fs.S3FileSystem()
    
    wm_mask_fname = f'{bucket_name}/sub-{subject}/sub-{subject}_wm_mask.nii.gz'
    if fs.exists(wm_mask_fname):
        log.info(f"WM mask exists. Reading from {wm_mask_fname}")
        wm_img = afd.s3fs_nifti_read(wm_mask_fname)
        wm_mask = wm_img.get_data()
    else:
        log.info(f"Calculating WM segmentation")
        wm_labels=[250, 251, 252, 253, 254, 255, 41, 2, 16, 77]
        seg_img = nib.load(op.join(anat_dir, f"sub-{subject}_aparc+aseg.nii.gz"))
        seg_data_orig = seg_img.get_fdata()
        # For different sets of labels, extract all the voxels that
        # have any of these values:
        wm_mask = np.sum(np.concatenate([(seg_data_orig == l)[..., None]
                                        for l in wm_labels], -1), -1)

        # Resample to DWI data:
        wm_mask = np.round(reg.resample(wm_mask, 
                                        dwi_data[..., 0],
                                        seg_img.affine,
                                        dwi_img.affine)).astype(int)

        wm_img = nib.Nifti1Image(wm_mask.astype(int),
                                 dwi_img.affine)
        log.info(f"Saving to {wm_mask_fname}")
        afd.s3fs_nifti_write(wm_img, wm_mask_fname)
        
    for model_object, method in zip([dti.TensorModel, dki.DiffusionKurtosisModel],
                                    ['dti', 'dki']):
        cod_fname = f'{bucket_name}/sub-{subject}/sub-{subject}_model-{method}_desc-cod.nii.gz'
        if fs.exists(cod_fname):
            log.info(f"{method} COD file alread exists. Skipping")
        else:
            log.info(f"Calculating {method}")
            model = model_object(gtab)
            pred = xval.kfold_xval(model, dwi_data, 5, mask=wm_mask)
            cod = xval.coeff_of_determination(pred, dwi_data)
            cod_img = nib.Nifti1Image(cod, dwi_img.affine)
            log.info(f"Writing COD to {cod_fname}")
            afd.s3fs_nifti_write(cod_img, cod_fname)

In [3]:
import configparser
import os.path as op

In [4]:
CP = configparser.ConfigParser()
CP.read_file(open(op.join(op.expanduser('~'), '.aws', 'credentials')))
CP.sections()
ak = CP.get('hcp', 'AWS_ACCESS_KEY_ID')
sk = CP.get('hcp', 'AWS_SECRET_ACCESS_KEY')

In [5]:
hcp_dki_dti_knot = ck.Knot(name='hcp_dki_dti_20200318_09',
                           func=dki_dti_cod,
                           base_image='python:3.7',
                           image_github_installs="https://github.com/yeatmanlab/pyAFQ.git@0.2",
                           pars_policies=('AmazonS3FullAccess',),
                           resource_type="SPOT",
                           bid_percentage=100,
                           memory=64000)

In [6]:
subjects = [100206,
            100307
           ]

In [7]:
result = hcp_dki_dti_knot.map([(s, ak, sk) for s in subjects])

In [8]:
j0 = hcp_dki_dti_knot.jobs[0]

In [11]:
j0.status

{'status': 'SUCCEEDED',
 'statusReason': None,
 'attempts': [],
 'arrayProperties': {'statusSummary': {'STARTING': 0,
   'FAILED': 0,
   'RUNNING': 0,
   'SUCCEEDED': 2,
   'RUNNABLE': 0,
   'SUBMITTED': 0,
   'PENDING': 0},
  'size': 2}}

In [10]:
# hcp_dki_dti_knot.clobber()